In [7]:
import pandas as pd
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict
import gensim
import datetime
from datetime import datetime as dt, timedelta as td
import re
import textstat
from textstat.textstat import textstat
import nltk
from itertools import chain
import pandasql as ps
# from BaseAD import CreateBaseAD,QuarterIndex ,get_quarteryr
# from qaextraction import get_analysts_from_preamble
import pandas as pd
import itertools
import time
from utils import get_date_from_preamble
import gc
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## Read Data From Output of join script 

set1 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_1.csv',sep='|')
set2 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_2.csv',sep='|')
set3 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_3.csv',sep='|')
set4 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_4.csv',sep='|')                  
# set2 = pd.read_csv('data/transcript_with_cals_1901_V1_2.csv',sep='|')
# set3 = pd.read_csv('data/transcript_with_cals_1901_V1_3.csv',sep='|')
# set4 = pd.read_csv('data/transcript_with_cals_1901_V1_4.csv',sep='|')


data_raw = set1.append([set2, set3, set4])

qa_len_list= [len(set1), len(set2), len(set3), len(set4)]

In [4]:
## Dropping intermediate files and clearing cache

del set1
del set2
del set3
del set4

gc.collect()

5402

In [8]:
np.sum(qa_len_list)

48844

In [9]:
data_raw["date_earning_call"] = data_raw["date_earning_call"].apply(pd.to_datetime)
data_raw["date_earning_call"] = data_raw["date_earning_call"].apply(pd.Timestamp.date)

In [11]:
data_raw = data_raw[['input_company_name2', 'About',
       'Transcript_page_heading', 'Transcript_heading', 'Preamble',
       'transcript_after_preamble', 'date_earning_call', 
       'year', 'published_date_time', 'next_publish_date', 'def_date',
       'day_diff', 'cal_def_value', 'filing_date_2', 'is_cals']]


data_raw = data_raw.drop_duplicates()

data_raw = data_raw[~data_raw['Preamble'].isnull()]
len(data_raw)

48844

In [16]:
#EDA KIND
# Create key at date and company level

#data_raw['date_earning_call_str'] = [str(i) for i in list(data_raw['date_earning_call'])]

data_raw['date_earning_call_str'] = data_raw['date_earning_call'].apply(str)
data_raw['key'] = data_raw['input_company_name2'] + data_raw['date_earning_call_str']
data_raw['Count'] = data_raw.groupby('key')['key'].transform('count')
#data_raw['max_published_date'] = data_raw.groupby('key')['published_date_time'].transform('max')

In [9]:
#Remove duplicates at earning date and compnay level

## Step 1 - Crete a subset of data whicch are having multiple counts at Key level 
data_temp = data[data['Count']>1 ]

## Step 2 - Records where published date is equal to maximum, Helps in removing instances where multiple edits/publishes has been done for trancripts
data_temp2 = data_temp[data_temp['published_date_time'] == data_temp['max_published_date']]

## Step 3 - Records which are having just one record at Key level and are not creating issues
data_temp1 = data[data['Count']==1]

## Append data from Step3 and Step2
data = data_temp1.append(data_temp2,ignore_index =True)

In [12]:
data[['input company name','About','Transcript heading','Transcript page heading', 'Date published', 'Preamble',
       'transcript after preamble', 'Transcript link','date_earning_call']].to_csv('D:\\Chubb\\Data\\Intermediate\\481_companies_earning_date.csv')
